In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import time 
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.common.by import By
import os

### 1. ChromeDriver 로 네이버호텔 열기

In [2]:
chrome_ver =chromedriver_autoinstaller.get_chrome_version().split('.')[0]
print(chrome_ver)

114


In [3]:
try :
    crawler = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')
except :
    chromedriver_autoinstaller.install(True)
    crawler = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')
crawler.implicitly_wait(10)
crawler.get('https://hotels.naver.com/')

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_8732\3010569745.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  crawler = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')


In [4]:
crawler.find_element(By.XPATH, r'//*[@id="__next"]/div/div[2]/div/div[2]/button[2]').click()
crawler.find_element(By.XPATH, r'//*[@id="__next"]/div/div[1]/div[2]/div/div/div/div[1]/button').click()
engine = crawler.find_element(By.XPATH, r'//*[@id="__next"]/div/div[2]/div[1]/div/input')
engine.send_keys('제주')

In [5]:
crawler.find_element(By.XPATH, r'//*[@id="__next"]/div/div[2]/div[2]/section/ul/li[2]/a/div/b/mark').click()
crawler.find_element(By.XPATH, r'//*[@id="__next"]/div/div[1]/div[2]/div/div/button').click()

### 2. 페이지 스크롤하며 Data 가져오기

In [7]:
base_url = 'https://hotels.naver.com/list?placeFileName=place%3AJeju_Province&adultCnt=2&includeTax=false&sortField=popularityKR&sortDirection=descending&pageIndex={}'

total = []

for i in range(-1,130):
    url = base_url.format(i+1)
    crawler.get(url)
    #print(i)
    while True:
        bh = crawler.execute_script("return document.body.scrollHeight")
        print(bh)
        time.sleep(4)
        crawler.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(2)
        ah = crawler.execute_script("return document.body.scrollHeight")
        if ah == bh:
            break
        bh = ah
    infos = crawler.find_elements(By.CSS_SELECTOR,'.Detail_InfoArea__uZ4qT' )
    for info in infos:
        try:
            name = info.find_element(By.CSS_SELECTOR,'.Detail_title__40_dz').text
            # names.append(name)
            prov = info.find_element(By.CSS_SELECTOR,'.Detail_location__u3_N6').text.strip(', 대한민국')
            # provs.append(prov)
            score = info.find_element(By.CSS_SELECTOR, '.Detail_score__UxnqZ').text
            # scores.append(score)
            grade = info.find_element(By.CSS_SELECTOR, '.Detail_grade__y5BmJ').text
            # grades.append(grade)
            total.append([name, prov,score,grade])
            
        except:
            print("Exception")
crawler.close()
            
        
    


4486
4828
Exception
4486
Exception
4431
4486
Exception
Exception
Exception
Exception
Exception
4486
Exception
4486
Exception
Exception
4486
Exception
Exception
Exception
Exception
Exception
Exception
4486
Exception
Exception
Exception
4486
Exception
Exception
Exception
Exception
4486


### 3. DF 생성 및 저장

In [ ]:
df = pd.DataFrame(total, columns=['name', 'prov', 'score', 'grade'])
print(df)

In [ ]:
df.to_csv("c:/프로젝트/Hotel Review/Hotel_info_all.csv", sep=',', encoding="utf-8-sig")